In [22]:
import torch
from transformers import T5Tokenizer
print(torch.__version__)


2.6.0+cpu


In [18]:
import pdfplumber
from transformers import pipeline
import nltk

# Download necessary NLTK data for sentence tokenization
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

# Define paths
pdf_path = r"C:\Users\gurur\OneDrive\Documents\python\code\Hands-On Machine Learning with Scikit-Learn and TensorFlow_ Concepts, Tools, and Techniques to Build Intelligent Systems ( PDFDrive ).pdf"
output_text_file = "extracted_text.txt"

# Load the summarization and question generation pipelines
summarizer = pipeline("summarization", model="t5-small")
qg_pipeline = pipeline("text2text-generation", model="t5-small")

# Function to extract text from the PDF
def extract_pdf_text(pdf_path):
    document_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                document_text += text + "\n"
    return document_text

# Extract text from the PDF
document_text = extract_pdf_text(pdf_path)

# Save extracted text to a file
with open(output_text_file, "w", encoding="utf-8") as text_file:
    text_file.write(document_text)

# Print preview of the extracted text
print("Preview of extracted text:")
print(document_text[:2000])

# Tokenize text into sentences
sentences = sent_tokenize(document_text)

# Combine sentences into passages (max 200 words per passage)
passages = []
current_passage = ""
for sentence in sentences:
    if len(current_passage.split()) + len(sentence.split()) < 200:  # Adjust limit if necessary
        current_passage += " " + sentence
    else:
        passages.append(current_passage.strip())
        current_passage = sentence
if current_passage:
    passages.append(current_passage.strip())

# Function to generate questions from a passage
def generate_questions_pipeline(passage, min_questions=3):
    input_text = f"generate questions: {passage}"
    results = qg_pipeline(input_text)
    questions = results[0]['generated_text'].split('<sep>')
    
    # Remove duplicates and empty strings
    questions = [q.strip() for q in questions if q.strip()]
    
    # If fewer than required questions, generate more from smaller parts
    if len(questions) < min_questions:
        passage_sentences = passage.split('. ')
        for i in range(len(passage_sentences)):
            if len(questions) >= min_questions:
                break
            additional_input = ' '.join(passage_sentences[i:i+2])
            additional_results = qg_pipeline(f"generate questions: {additional_input}")
            additional_questions = additional_results[0]['generated_text'].split('<sep>')
            questions.extend([q.strip() for q in additional_questions if q.strip()])
    
    return questions[:min_questions]  # Return only the top questions

# Generate questions from passages and answer them
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

def answer_unique_questions(passages, qa_pipeline):
    answered_questions = set()  # Store answered questions to avoid repetition

    for idx, passage in enumerate(passages):
        print(f"Passage {idx + 1}:\n{passage}\n")
        questions = generate_questions_pipeline(passage)
        
        for question in questions:
            if question not in answered_questions:
                # Get answer using the QA pipeline
                answer = qa_pipeline({'question': question, 'context': passage})
                print(f"Q: {question}")
                print(f"A: {answer['answer']}\n")
                answered_questions.add(question)  # Avoid answering the same question again
        
        print(f"{'='*50}\n")

# Call the function to generate and answer unique questions from the passages
answer_unique_questions(passages, qa_pipeline)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gurur\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Device set to use cpu
Device set to use cpu


Preview of extracted text:
Hands-On
Machine Learning
with Scikit-Learn
& TensorFlow
CONCEPTS, TOOLS, AND TECHNIQUES
TO BUILD INTELLIGENT SYSTEMS
Aurélien Géron
Hands-On Machine Learning with
Scikit-Learn and TensorFlow
Concepts, Tools, and Techniques to
Build Intelligent Systems
Aurélien Géron
BBeeiijjiinngg BBoossttoonn FFaarrnnhhaamm SSeebbaassttooppooll TTookkyyoo
Hands-On Machine Learning with Scikit-Learn and TensorFlow
by Aurélien Géron
Copyright © 2017 Aurélien Géron. All rights reserved.
Printed in the United States of America.
Published by O’Reilly Media, Inc., 1005 Gravenstein Highway North, Sebastopol, CA 95472.
O’Reilly books may be purchased for educational, business, or sales promotional use. Online editions are
also available for most titles (http://oreilly.com/safari). For more information, contact our corporate/insti‐
tutional sales department: 800-998-9938 or corporate@oreilly.com.
Editor: Nicole Tache Indexer: Wendy Catalano
Production Editor: Nicholas Adams Interior

Device set to use cpu


Passage 1:
Hands-On
Machine Learning
with Scikit-Learn
& TensorFlow
CONCEPTS, TOOLS, AND TECHNIQUES
TO BUILD INTELLIGENT SYSTEMS
Aurélien Géron
Hands-On Machine Learning with
Scikit-Learn and TensorFlow
Concepts, Tools, and Techniques to
Build Intelligent Systems
Aurélien Géron
BBeeiijjiinngg BBoossttoonn FFaarrnnhhaamm SSeebbaassttooppooll TTookkyyoo
Hands-On Machine Learning with Scikit-Learn and TensorFlow
by Aurélien Géron
Copyright © 2017 Aurélien Géron. All rights reserved. Printed in the United States of America. Published by O’Reilly Media, Inc., 1005 Gravenstein Highway North, Sebastopol, CA 95472. O’Reilly books may be purchased for educational, business, or sales promotional use. Online editions are
also available for most titles (http://oreilly.com/safari). For more information, contact our corporate/insti‐
tutional sales department: 800-998-9938 or corporate@oreilly.com. Editor: Nicole Tache Indexer: Wendy Catalano
Production Editor: Nicholas Adams Interior Designer: David

Token indices sequence length is longer than the specified maximum sequence length for this model (529 > 512). Running this sequence through the model will result in indexing errors


Q: questions: 7 It’s okay if you don’t understand all the code yet;
A: 



Passage 54:
Training and running a linear model using Scikit-Learn
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
# Load the data
oecd_bli = pd.read_csv("oecd_bli_2015.csv", thousands=',')
gdp_per_capita = pd.read_csv("gdp_per_capita.csv",thousands=',',delimiter='\t',
encoding='latin1', na_values="n/a")
# Prepare the data
country_stats = prepare_country_stats(oecd_bli, gdp_per_capita)
X = np.c_[country_stats["GDP per capita"]]
y = np.c_[country_stats["Life satisfaction"]]
# Visualize the data
country_stats.plot(kind='scatter', x="GDP per capita", y='Life satisfaction')
plt.show()
# Select a linear model
lin_reg_model = sklearn.linear_model.LinearRegression()
# Train the model
lin_reg_model.fit(X, y)
# Make a prediction for Cyprus
X_new = [[22587]] # Cyprus' GDP per capita
print(lin_reg_model.predict(X_new)) # outputs [[ 5.96242338]]
If you had used an inst

KeyboardInterrupt: 